In [1]:
from keras.datasets import cifar10
from keras.models import Sequential,load_model
from keras.layers import Conv2D,ZeroPadding2D,Activation,MaxPooling2D,Dense,Flatten,Dropout,BatchNormalization
from keras.regularizers import l2
from keras.utils import np_utils
from keras.optimizers import Adadelta

Using TensorFlow backend.


In [2]:
(x_train,y_train),(x_test,y_test) = cifar10.load_data()

print('x train shape ',x_train.shape)
print(x_train.shape[0],' training samples')
print(x_test.shape[0],' test samples')

170500096/170498071 [==============================] - 4s 0us/step
x train shape  (50000, 32, 32, 3)
50000  training samples
10000  test samples


**The CIFAR-10 dataset**
The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images.

The dataset is divided into five training batches and one test batch, each with 10000 images. The test batch contains exactly 1000 randomly-selected images from each class. The training batches contain the remaining images in random order, but some training batches may contain more images from one class than another. Between them, the training batches contain exactly 5000 images from each class.

In [3]:
num_classes = 10
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

In [7]:
l2_reg = 0
#let's replicate AlexNet
model = Sequential()

#layer 1
model.add(Conv2D(96,(11,11),input_shape = x_train.shape[1:],
                 padding='same',kernel_regularizer = l2(l2_reg)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D((2,2)))

#layer2
model.add(Conv2D(256,(5,5),padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D((2,2)))

#layer3
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512,(3,3),padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D((2,2)))

#layer4
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(1024,(3,3),padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

#layer5
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(1024,(3,3),padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D((2,2)))

# 1st FC layer
model.add(Flatten())
model.add(Dense(3072))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

# 2nd FC layer
model.add(Dense(4096))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

# 3rd FC layer
model.add(Dense(num_classes))
model.add(BatchNormalization())
model.add(Activation('softmax'))


model.compile(loss = 'categorical_crossentropy',
              optimizer = Adadelta(),
              metrics = ['accuracy'])

#model.save('Alexnet_implementation_cifar10.h5')

In [8]:
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 96)        34944     
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 96)        384       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 96)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 96)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 256)       614656    
_________________________________________________________________
batch_normalization_2 (Batch (None, 16, 16, 256)       1024      
_________________________________________________________________
activation_2 (Activation)    (None, 16, 16, 256)      

In [9]:
batch_size=32
epochs = 1
history = model.fit(x_train,y_train,
                    batch_size = batch_size,
                    epochs = epochs,
                    validation_data = (x_test,y_test),
                    shuffle = True)

scores = model.evaluate(x_test,y_test,verbose = 1)
print('test val loss : ',score[0])
print('test val accuracy :',score[1])

Train on 50000 samples, validate on 10000 samples
Epoch 1/1
 2976/50000 [>.............................] - ETA: 2:51:29 - loss: 1.9691 - accuracy: 0.2802

KeyboardInterrupt: ignored